## Read alignment problem. Finds proportion of sequecing reads matching with the reference genome

In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq #sequencing read

--2023-07-09 00:47:37--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.161.245.142, 18.161.245.3, 18.161.245.40, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.161.245.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: ‘ERR266411_1.first1000.fastq’

ERR266411_1.first10 100%[===================>] 248.42K   410KB/s    in 0.6s    

2023-07-09 00:47:40 (410 KB/s) - ‘ERR266411_1.first1000.fastq’ saved [254384/254384]



In [2]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa #viral genome

--2023-07-09 00:47:44--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.161.245.142, 18.161.245.40, 18.161.245.3, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.161.245.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa’

phix.fa             100%[===================>]   5.40K  --.-KB/s    in 0s      

2023-07-09 00:47:45 (92.7 MB/s) - ‘phix.fa’ saved [5528/5528]



In [3]:
def naiveMatch(P,T):
    #returns positions at which a read matches the genome
    offsets = []
    
    for i in range(len(T) - len(P) + 1):
        match = True
        
        for j in range(len(P)):
            if T[i+j] != P[j]:
                match = False
                break
        if match:
            offsets.append(i)
        
    return  offsets

def readFasta(filename):
    #reads a fasta file a returns a string of nucleotides from it
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip() #removes white space or new line character
    return  genome

import random
def generateReads(genome, numReads, readLen):
    '''Generate reads from random positions in the given genome.'''
    
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome) - readLen)
        reads.append(genome[start : start + readLen])
    return reads

def readFastq(filename):
    #reads a fastq- sequencing output file - and returns two lists of sequence and quality
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().strip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0: #check if the loop reached end of the file
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities


def revComplement(s):
    #gives reverse compliment of a nucleotide strin. N represents
    #an unsure nucleotide from the sequecing data
    complement = {'A':'T','T':'A','G':'C','C':'G', 'N':'N'}

    rev_comp = ''
    for base in s:
        rev_comp = complement[base] + rev_comp
        
    return rev_comp



In [4]:
phix_reads, _ = readFastq('ERR266411_1.first1000.fastq')
genome = readFasta('phix.fa')
#reads = generateReads(genome, 100, 100)

In [5]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]
    matches = naiveMatch(r,genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
        
print('%d / %d reads matched exactly' % (numMatched, n))

459 / 1000 reads matched exactly


7/1000 is too low. it could be due to sequencing errors, or due to the difference in reference genome and actual genome.
Next we reduce the read length to 30 from 100. it gave 459/1000 reads matched. 
Another reason why the matches are low even tho reads are from the same genome (matched against reference genome) is because the genome is double stranded but the reads can be from any of the two, but our programe only looks on one the strands, its not looking on both. we should not only match the read to the genome but also match reverse complement of the reads to the genome.

In [6]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]
    matches = naiveMatch(r,genome)
    matches.extend(naiveMatch(revComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
        
print('%d / %d reads matched exactly' % (numMatched, n))

932 / 1000 reads matched exactly


When reverse complement is also matched 932/1000 reads matched with the genome.